In [ ]:
%pip install llama-index  llama-index-llms-cohere llama-index-embeddings-cohere llama_agents langchain_community duckduckgo_search

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
# Initilise with your api key
import os

cohere_api_key = ""
os.environ["COHERE_API_KEY"] = cohere_api_key


In [5]:
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.llms.cohere import Cohere
from llama_index.core import Settings

llm = Cohere(api_key=cohere_api_key, model="command-r-plus")

# with input_typ='search_query'
embed_model = CohereEmbedding(
    api_key=cohere_api_key,
    model_name="embed-english-v3.0",
    input_type="search_query",
)
# global
Settings.embed_model = embed_model
Settings.llm = llm


In [6]:
from llama_index.core import  ServiceContext

# Create the service context with the cohere model for generation and embedding model
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model
)

<ipython-input-6-45a44ac845c6>:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [8]:
from llama_index.core import  SimpleDirectoryReader

docs = SimpleDirectoryReader("/content/data").load_data()

In [9]:
import os

from llama_index.core import (
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)

# build index
index = VectorStoreIndex.from_documents(docs ,service_context=service_context )
retriever = index.as_retriever()

In [11]:
from langchain_community.tools import DuckDuckGoSearchResults
from llama_index.core.tools import FunctionTool

def duckduckgo_search(query:str) -> str :
  search = DuckDuckGoSearchResults(max_results=15)
  return search.run(query)

In [12]:
from llama_agents import ComponentService, ServiceComponent, SimpleMessageQueue

message_queue = SimpleMessageQueue()


def to_service_component(component, message_queue, service_name, description):
    server = ComponentService(
        component=component,
        message_queue=message_queue,
        description=description,
        service_name=service_name,
    )
    service_component = ServiceComponent.from_component_service(server)
    return service_component, server

In [13]:
from llama_index.core.prompts import PromptTemplate
from llama_index.core.query_pipeline import QueryPipeline

relevancy_prompt_tmpl = PromptTemplate(
    template="""As a grader, your task is to evaluate the relevance of a document retrieved in response to a user's question.

    Retrieved Document:
    -------------------
    {context_str}

    User Question:
    --------------
    {query_str}

    Evaluation Criteria:
    - Consider whether the document contains keywords or topics related to the user's question.
    - The evaluation should not be overly stringent; the primary objective is to identify and filter out clearly irrelevant retrievals.

    Decision:
    - Assign a binary score to indicate the document's relevance.
    - Use 'yes' if the document is relevant to the question, or 'no' if it is not.

    Please provide your binary score ('yes' or 'no') below to indicate the document's relevance to the user question."""
)
relevancy_qp = QueryPipeline(chain=[relevancy_prompt_tmpl, llm])

In [14]:
# define RAG agent
from llama_index.core.query_pipeline import FnComponent
from typing import Dict


def run_retrieval(input_str: str) -> Dict:
    """Run Retrieval."""
    # retrieves a set of nodes
    retrieved_nodes = retriever.retrieve(input_str)

    # runs a relevancy check
    relevancy_results = []
    for node in retrieved_nodes:
        relevancy = relevancy_qp.run(context_str=node.text, query_str=query_str)
        relevancy_results.append(relevancy.message.content.lower().strip())
    contains_irrelevant = "no" in relevancy_results

    # get relevant texts
    relevant_texts = [
        retrieved_nodes[i].text
        for i, result in enumerate(relevancy_results)
        if result == "yes"
    ]
    relevant_text = "\n".join(relevant_texts)

    # returns a dictionary of items
    return {
        "relevant_text": relevant_text,
        "contains_irrelevant": contains_irrelevant,
        "input_str": input_str,
    }


retrieval_component = FnComponent(fn=run_retrieval)
retrieval_component_s, retrieval_server = to_service_component(
    retrieval_component,
    message_queue,
    "Runs a retrieval + relevancy check",
    "retrieval_service",
)

In [15]:
from llama_index.core.prompts import PromptTemplate

query_transform_tmpl = PromptTemplate(
    template="""Your task is to refine a query to ensure it is highly effective for retrieving relevant search results. \n
    Analyze the given input to grasp the core semantic intent or meaning. \n
    Original Query:
    \n ------- \n
    {query_str}
    \n ------- \n
    Your goal is to rephrase or enhance this query to improve its search performance. Ensure the revised query is concise and directly aligned with the intended search objective. \n
    Respond with the optimized query only:"""
)
query_transform_qp = QueryPipeline(chain=[query_transform_tmpl, llm])

In [16]:
def run_web_search(input_str: str) -> str:
    """Run Web Search."""

    transformed_query_str = query_transform_qp.run(query_str=input_str).message.content
    # Conduct a search with the transformed query string and collect the results.
    search_results = duckduckgo_search(transformed_query_str)
    return search_results


web_search_component = FnComponent(fn=run_web_search)
web_search_component_s, web_server = to_service_component(
    web_search_component, message_queue, "Runs web search", "web_search_service"
)

In [17]:
from llama_index.core.base.response.schema import Response
from llama_index.core.schema import Document
from llama_index.core import SummaryIndex
from typing import Optional


def run_summarization(retrieved_text: str, search_text: Optional[str] = None) -> str:
    """Run summarization."""
    # use summary index to perform summarization
    search_text = search_text or ""
    documents = [Document(text=retrieved_text + "\n" + search_text)]
    index = SummaryIndex.from_documents(documents)
    query_engine = index.as_query_engine()
    return str(query_engine.query(query_str))


summary_component = FnComponent(fn=run_summarization)
summary_component_s, summary_server = to_service_component(
    summary_component, message_queue, "Run summarization", "summarization_service"
)

In [18]:
from llama_agents import (
    AgentService,
    ControlPlaneServer,
    SimpleMessageQueue,
    PipelineOrchestrator,
    ServiceComponent,
    ComponentService,
)
from llama_index.core.query_pipeline import Link, InputComponent

pipeline = QueryPipeline(
    module_dict={
        "input": InputComponent(),
        "retrieval_server": retrieval_component_s,
        "web_server": web_search_component_s,
        "summary_server_no_web": summary_component_s,
        "summary_server_web": summary_component_s,
    }
)
pipeline.add_link("input", "retrieval_server")
pipeline.add_link(
    "retrieval_server",
    "web_server",
    condition_fn=lambda x: x["contains_irrelevant"],
    input_fn=lambda x: x["input_str"],
)
# if web search is called
pipeline.add_link(
    "retrieval_server",
    "summary_server_web",
    dest_key="retrieved_text",
    condition_fn=lambda x: x["contains_irrelevant"],
    input_fn=lambda x: x["relevant_text"],
)
pipeline.add_link("web_server", "summary_server_web", dest_key="search_text")

# if web search is not called
pipeline.add_link(
    "retrieval_server",
    "summary_server_no_web",
    dest_key="retrieved_text",
    condition_fn=lambda x: not x["contains_irrelevant"],
    input_fn=lambda x: x["relevant_text"],
)

pipeline_orchestrator = PipelineOrchestrator(pipeline)

control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=pipeline_orchestrator,
)

In [30]:
from llama_agents.launchers import LocalLauncher

## Define Launcher
launcher = LocalLauncher(
    [retrieval_server, web_server, summary_server],
    control_plane,
    message_queue,
)

In [ ]:
while True:
    query_str = input("Enter query: ")
    result = launcher.launch_single(query_str)
    print(str(result))